In [ ]:
# %env USING_RUN True
%run /work/greenhouse-simulator-2/greenhouse/adaptive_lighting/LED_lighting.ipynb import LED_lighting
%run /work/greenhouse-simulator-2/greenhouse/adaptive_lighting/solarpanel.ipynb import SolarPanel
%run /work/greenhouse-simulator-2/crops/sweet_basil.ipynb import SweetBasil

import sys
sys.path.insert(0, '/work/greenhouse-simulator-2/')

from helpers.types import *
from helpers.data_prep import *
from helpers.conversions import *
from helpers.visualization import *

In [ ]:
class AdaptiveLighting:
    def __init__(self, time_period: s, structure, crop):
        self.time_period = time_period
        self.structure = structure
        self.crop = crop
        self.light = LED_Lighting(barrel_count=self.structure.barrel_count)
        self.solarpanel = SolarPanel(
            time_period=self.time_period, 
            photoperiod=self.crop.photoperiod, 
            target_DLI=self.crop.target_DLI, 
            irradiated_area=self.structure.irradiated_area,
            # manufacturer="Insolight"
            # manufacturer="SolarBrite"
            manufacturer="NonSolar"
        )

    def is_dark_hour(self, hour_of_day, photoperiod):
        # Dark hours start at:
        start_at = 21
        is_dark = (hour_of_day >= start_at) or (hour_of_day < (24 - (24 - (start_at + 1)) - photoperiod))
        return is_dark


    def run(self, timestamp, natural_PAR_inside: umol_per_m2):
        """
        Calculates light supplementation, resulting in energy used for it and wasted/actual PAR as umol_per_m2
        """
        target_PAR_per_hour: umol_per_m2_hour = 1e6 * self.crop.target_DLI / self.crop.photoperiod

        # Get the current target DLI, which is either the target DLI, or 0 during the night
        target_PAR_current_hour: umol_per_m2_hour = target_PAR_per_hour
        if self.is_dark_hour(timestamp.hour, self.crop.photoperiod):
            target_PAR_current_hour = 0

        # Get target and natural PAR as amount of photons
        target_PAR: umol_per_m2 = target_PAR_current_hour * (self.time_period / 3600)
        target_PAR_total: umol = target_PAR * self.structure.barrel_surface_total
        natural_PAR_total: umol = natural_PAR_inside * self.structure.barrel_surface_exposed_to_sun

        # Natural light is not enough, supplement needed
        supplemented_PAR_total: umol = 0
        energy_used_for_lighting_kWh: kWh = 0
        if target_PAR_total > natural_PAR_total:
            PAR_to_supplement = target_PAR_total - natural_PAR_total
            supplemented_PAR_total += PAR_to_supplement

            light_results: [J, J] = self.light.run(
                timestamp,
                self.crop.photoperiod,
                self.time_period,
                self.PAR_total_to_PPFD(PAR_to_supplement)
            )
            energy_used_for_lighting, heat_generated_from_lighting = light_results
            energy_used_for_lighting_kWh: kWh = J_to_kWh(energy_used_for_lighting)

        # Natural light is too much, PAR above target is wasted
        wasted_PAR_total: umol = 0
        if target_PAR_total < natural_PAR_total:
            wasted_PAR_total = natural_PAR_total - target_PAR_total

        return {
            "energy_used_for_lighting_kWh": energy_used_for_lighting_kWh,
            "wasted_PAR_total_umol": wasted_PAR_total,
            "wasted_PPFD_umol_per_m2_s": self.PAR_total_to_PPFD(wasted_PAR_total),
            "actual_PPFD_umol_per_m2_s": self.PAR_total_to_PPFD(natural_PAR_total + supplemented_PAR_total),
            "supplemented_PPFD_umol_per_m2_s": self.PAR_total_to_PPFD(supplemented_PAR_total),
            "natural_PPFD_umol_per_m2_s": self.PAR_total_to_PPFD(natural_PAR_total),
            "natural_PAR_total": natural_PAR_total,
            "supplemented_PAR_total": supplemented_PAR_total,
            "wasted_PAR_total": wasted_PAR_total
        }

    def PAR_total_to_PPFD(self, PAR_total: umol) -> umol_per_m2_s:
        return (PAR_total / self.structure.barrel_surface_total) / self.time_period

In [ ]:
plot = None
if not os.environ.get("USING_RUN"):
    resample_period = "60min"
    df = get_weather_data(date_from="2020-01-01", date_to="2020-12-31", resample_period=resample_period)

    time_period = pd.to_timedelta(resample_period).total_seconds()
    structure = Structure()
    crop = SweetBasil(time_period=time_period, plants_per_barrel=200, barrel_count=6)

    light = AdaptiveLighting(time_period=time_period, structure=structure, crop=crop)

    for timestamp, data in df.iterrows():
        results: [W, W_per_m2] = structure.get_irradiance_by_panel_type(timestamp, data.solarradiation)
        _, irradiance_on_solar_panels = results

        results: [W, W, W_per_m2] = light.solarpanel.run(irradiance_on_solar_panels)
        solar_power_generated, _, transmitted_irradiance, transparency, _ = results
        solar_energy_generated: kWh = J_to_kWh(solar_power_generated * time_period)

        PPFD: umol_per_m2_s = irradiance_to_PPFD(transmitted_irradiance)
        natural_PAR_inside: umol_per_m2 = PPFD * time_period

        results = light.run(timestamp, natural_PAR_inside)

        results["transparency"] = transparency
        results["solar_energy_generated_kWh"] = solar_energy_generated

        for col in results.keys():
            df.loc[timestamp, col] = results[col]


    stacked_chart = plot_stacked_area(
        df, 
        ["natural_PPFD_umol_per_m2_s", "supplemented_PPFD_umol_per_m2_s"],
        width=900, 
        height=235, 
        title="", 
        y_label="PPFD (umol / m2 s)", 
        legend_label="", 
        date_format="",
        colors=["#eb9b34", "#ebe134"]
    )

    line_chart = plot_multiline(
        df, 
        ["transparency"],
        width=900, 
        height=235, 
        title="", 
        y_label="", 
        legend_label="", 
        date_format="",
        colors=["#333"]
    )

    total_energy_generated: kWh = df["solar_energy_generated_kWh"].sum()
    total_energy_used: kWh = df["energy_used_for_lighting_kWh"].sum()


    print(f"Net energy: {total_energy_generated - total_energy_used} kWh")

    plot = alt.layer(stacked_chart, line_chart).resolve_scale(y='independent', color='independent') 

plot

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=2f5dc715-67f7-4c8c-98f7-a87b736d3338' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>